## Heroes of Pymoli

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# Raw data file
csv_file = "Resources/purchase_data.csv"

# Read data file and store into data frame
purchase_data = pd.read_csv(csv_file)

In [3]:
# Clean/Prepare the data
item_data = purchase_data.loc[:,["Item ID", "Item Name", "Price"]]
player_demographics = purchase_data.loc[:, ["Gender", "SN", "Age"]]
player_demographics = player_demographics.drop_duplicates()

In [4]:
# Player Count

# Do the math
player_count = player_demographics.count()[0]

# Make pretty output
player_count_data = pd.DataFrame({"Total Players": [player_count]})

### Player Count

In [5]:
player_count_data

,Total Players
0,576


In [12]:
# Purchasing Analysis (Total)

# Do the math
item_count = len(purchase_data["Item ID"].unique())
average_purchase_price = purchase_data["Price"].mean()
purchase_count = purchase_data["Price"].count()
total_revenue = purchase_data["Price"].sum()

In [13]:
# Save to data frame
purchasing_analysis = pd.DataFrame({"Number of Unique Items": item_count,
                                    "Average Purchase Price": average_purchase_price,
                                    "Total Number of Purchases": purchase_count,
                                    "Total Revenue": total_revenue}, index=[0])

In [14]:
# Make pretty output
purchasing_analysis_data = purchasing_analysis.round(2)
purchasing_analysis_data["Average Purchase Price"] = purchasing_analysis_data["Average Purchase Price"].map("${:,.2f}".format)
purchasing_analysis_data["Total Number of Purchases"] = purchasing_analysis_data["Total Number of Purchases"].map("${:,.2f}".format)
purchasing_analysis_data["Total Revenue"] = purchasing_analysis_data["Total Revenue"].map("${:,.2f}".format)

### Purchasing Analysis (Total)

In [15]:
purchasing_analysis_data

,Number of Unique Items,Average Purchase Price,Total Number of Purchases,Total Revenue
0,183,$3.05,$780.00,"$2,379.77"


In [16]:
# Gender Demographics

# Do the math
gender_demographics_totals = player_demographics["Gender"].value_counts()
gender_demographics_percents = gender_demographics_totals / player_count * 100

# Make pretty output
gender_demographics_data = pd.DataFrame({"Total Count": gender_demographics_totals,
                                         "Percentage of Players": gender_demographics_percents}).round(2)

### Gender Demographics

In [17]:
gender_demographics_data

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [18]:
# Purchasing Analysis (Gender)

# Do the math
gender_purchase_total = purchase_data.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
gender_purchase_average = purchase_data.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
gender_purchase_counts = purchase_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")
gender_purchase_average_per_person = gender_purchase_total / gender_demographics_data["Total Count"]

# Make pretty output
gender_purchase_data = pd.DataFrame({"Purchase Count": gender_purchase_counts,
                                     "Average Purchase Price": gender_purchase_average.map("${:,.2f}".format),
                                     "Total Purchase Value": gender_purchase_total.map("${:,.2f}".format),
                                     "Average Per Person": gender_purchase_average_per_person.map("${:,.2f}".format)})

### Purchasing Analysis (Gender)

In [19]:
gender_purchase_data

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [20]:
# Age Demographics

# Do the math
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
age_groups_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

player_demographics["Age Ranges"] = pd.cut(player_demographics["Age"], age_bins, labels=age_groups_names)

age_demographics_totals = player_demographics["Age Ranges"].value_counts()
age_demographics_percents = age_demographics_totals / player_count * 100

# Make pretty output
age_demographics_data = pd.DataFrame({"Total Count": age_demographics_totals,
                                      "Percentage of Players": age_demographics_percents.round(2)})

### Age Demographics

In [21]:
age_demographics_data.sort_index()

,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [22]:
# Purchasing Analysis (Age)

# Do the math
purchase_data["Age Ranges"] = pd.cut(purchase_data["Age"], age_bins, labels=age_groups_names)

age_purchase_total = purchase_data.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
age_purchase_average = purchase_data.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_purchase_counts = purchase_data.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")
age_purchase_average_per_person = age_purchase_total / age_demographics_data["Total Count"]

# Make pretty output
age_purchase_data = pd.DataFrame({"Purchase Count": age_purchase_counts,
                                  "Average Purchase Price": age_purchase_average.map("${:,.2f}".format),
                                  "Total Purchase Value": age_purchase_total.map("${:,.2f}".format),
                                  "Average Per Person": age_purchase_average_per_person.map("${:,.2f}".format)})

### Purchasing Analysis (Age)

In [23]:
age_purchase_data.sort_index()

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Per Person
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19
<10,23,$3.35,$77.13,$4.54


In [24]:
# Top Spenders

# Do the math
user_purchase_total = purchase_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
user_purchase_average = purchase_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_purchase_counts = purchase_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")

# Make pretty output
user_purchase_data = pd.DataFrame({"Purchase Count": user_purchase_counts,
                                   "Average Purchase Price": user_purchase_average.map("${:,.2f}".format),
                                   "Total Purchase Value": user_purchase_total.map("${:,.2f}".format)})

### Top Spenders

In [25]:
user_purchase_data.sort_values("Purchase Count", ascending=False).head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


In [26]:
# Most Popular Items

# Do the math
item_popular_purchase_total = item_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
item_popular_purchase_price = item_data.groupby(["Item ID", "Item Name"]).min()["Price"]
item_popular_purchase_counts = item_data.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")

# Make pretty output
item_popular_purchase_data = pd.DataFrame({"Purchase Count": item_popular_purchase_counts,
                                           "Item Price":item_popular_purchase_price.map("${:,.2f}".format),
                                           "Total Purchase Value": item_popular_purchase_total.map("${:,.2f}".format)})

### Most Popular Items

In [27]:
item_popular_purchase_data.sort_values("Purchase Count", ascending=False).head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


### Most Profitable Items

In [28]:
item_popular_purchase_data.sort_values("Total Purchase Value", ascending=False).head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
63,Stormfury Mace,2,$4.99,$9.98
29,"Chaos, Ender of the End",5,$1.98,$9.90
173,Stormfury Longsword,2,$4.93,$9.86
1,Crucifer,3,$3.26,$9.78
38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48


### Observable Trends

> Game players who are male between the ages of 20 and 24 tend to purchase more than any other age group.  Females spend has a higher per person purchase price than males.  Players between 30 and 39 have the highest per person purchase price.